In [13]:
from langchain.chains import TransformChain
import requests

In [8]:
import os
from dotenv import load_dotenv

# Load environment variables
if load_dotenv():
    print("Found Azure OpenAI API Base Endpoint: " + os.getenv("AZURE_OPENAI_ENDPOINT"))
else: 
    print("Azure OpenAI API Base Endpoint not found. Have you configured the .env file?")


Found Azure OpenAI API Base Endpoint: https://swed-oai.openai.azure.com/


In [9]:
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"
bing_subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")

In [29]:
# Open the file
with open('guest_name.txt', 'r') as f:
    guest = f.read()

# Print the content
print(guest)

Neil deGrasse Tyson


In [11]:
def bing_grounding(input_dict:dict) -> dict:
    print("Calling Bing Search API to get bio for guest...\n")
    search_term = input_dict["guest"]
    print("Search term is " + search_term)

    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()
    #print(search_results)

    # Parse out a bio.  
    bio = search_results["webPages"]["value"][0]["snippet"]
    
    print("Bio:\n")
    print(bio)
    print("\n")

    return {"bio": bio}


In [14]:
bing_chain = TransformChain(input_variables=["guest"], output_variables=["bio"], transform=bing_grounding)
bio = bing_chain.run(guest)


Calling Bing Search API to get bio for guest...

Search term is Neil deGrasse Tyson
Bio:

<b><b>Neil deGrasse Tyson</b></b> (US: / d ə ˈ ɡ r æ s / də-GRASS or UK: / d ə ˈ ɡ r ɑː s / də-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator<b>. <b>Tyso</b>n</b> studied at Harvard University , the University of Texas at Austin , and Columbia University .




In [15]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)

# Function calling in Assistance API


In [95]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [114]:
from openai import AzureOpenAI


client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version = "2023-12-01-preview"
)
model =  os.getenv("AZURE_OPENAI_CHAT_MODEL")
print(model)

gpt-35-turbo


In [126]:
bing_search_url = "https://api.bing.microsoft.com/v7.0/search"
bing_subscription_key = os.getenv("BING_SUBSCRIPTION_KEY")

In [127]:
import json
import requests
import time

from openai import AzureOpenAI
from pathlib import Path
from typing import Optional


def search_bing(query):
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_subscription_key}
    params = {"q": query, "textDecorations": False}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return json.dumps(output)
    

In [128]:
search_bing(guest)

'[{"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://en.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (US: / d \\u0259 \\u02c8 \\u0261 r \\u00e6 s / d\\u0259-GRASS or UK: / d \\u0259 \\u02c8 \\u0261 r \\u0251\\u02d0 s / d\\u0259-GRAHSS; born October 5, 1958) is an American astrophysicist, author, and science communicator. Tyson studied at Harvard University , the University of Texas at Austin , and Columbia University ."}, {"title": "Neil deGrasse Tyson - Wikipedia", "link": "https://nl.wikipedia.org/wiki/Neil_deGrasse_Tyson", "snippet": "Neil deGrasse Tyson (New York, 5 oktober 1958) is een Amerikaanse astrofysicus en wetenschapscommunicator. Hij houdt zich onder meer bezig met stervorming, exploderende sterren, dwergstelsels en de structuur van de Melkweg."}, {"title": "Neil deGrasse Tyson | Biography, Books, TV Shows, & Facts", "link": "https://www.britannica.com/biography/Neil-deGrasse-Tyson", "snippet": "Neil deGrasse Tyson, American astronome

In [136]:
messages = [{"role": "user", "content": "Provide the latest information about "+guest}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information about people by their names.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query contains people names",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

In [137]:
response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
        )

In [138]:
response_message = response.choices[0].message
tool_calls = response_message.tool_calls
print(response_message)
print(tool_calls)

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_VwS2D4W5DcBu7bCOrqBfv7qI', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson"\n}', name='search_bing'), type='function')])
[ChatCompletionMessageToolCall(id='call_VwS2D4W5DcBu7bCOrqBfv7qI', function=Function(arguments='{\n  "query": "Neil deGrasse Tyson"\n}', name='search_bing'), type='function')]


In [139]:
if tool_calls:
        # Step 3: call the function
        # Note: the JSON response may not always be valid; be sure to handle errors
        available_functions = {
            "search_bing": search_bing,
        }  # only one function in this example, but you can have multiple
        messages.append(response_message)  # extend conversation with assistant's reply
        # Step 4: send the info for each function call and function response to the model
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )  # extend conversation with function response
second_response = client.chat.completions.create(
            model=model,
            messages=messages,
        )  # get a new response from the model where it can see the function response

In [144]:
bio = second_response.choices[0].message.content
print(bio)

Neil deGrasse Tyson is an American astrophysicist, author, and science communicator. He was born on October 5, 1958. Tyson is known for his expertise in astrophysics, as well as his ability to popularize science through various media platforms.

He studied at Harvard University, the University of Texas at Austin, and Columbia University. Tyson has made significant contributions to the field of astrophysics, particularly in the areas of star formation, exploding stars, dwarf galaxies, and the structure of the Milky Way.

Tyson has hosted several popular science television series, including "Origins," "NOVA scienceNOW," and the revival of Carl Sagan's "Cosmos." He also hosts a radio show called "StarTalk."

As an author, Tyson has written several books, including "The Pluto Files: The Rise and Fall of America's Favorite Planet," which explores the controversy over the reclassification of Pluto as a dwarf planet. He continues to engage and educate the public through his books, television 

In [145]:
# Specify the file path
file_path = "bio.txt"

# Write the content to the file
with open(file_path, "w") as file:
    file.write(bio)